In [1]:
import time
import os
import math
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa
import shutil
import imageio
from PIL import Image, ImageDraw

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_DAT import AutoDAT
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ
from lib.analysis.laminar_dist import *
from lib.file.TIF import *
from lib.analysis.align import ImageAlign
from lib.utilities import *

In [2]:
# Make a series of 1-frame measure windows from PhotoZ

# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
google_drive_dir = "G:/.shortcut-targets-by-id/1afwzLrN3AQQPbJ1PQiZlKAb1Bijc5qln/Choosing trajectories/"
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/"
start_frame = 94
movie_length = 27

disable_photoZ_interact = False
disable_google_drive_interact = True
initialize_photoZ = False
clean_scratch_files = False  # clean up SNR maps
replace_existing_frame_pngs = False

In [3]:
# set up PhotoZ (open it manually)
aPhz = AutoPhotoZ(data_dir=data_dir)
if initialize_photoZ and not disable_photoZ_interact:
    aPhz.prepare_photoZ()
if not disable_photoZ_interact:
    aPhz.click_normalize_2_array_max()
pa.alert("Please disable trace overlay and maximize PhotoZ before continuing.")

'OK'

In [4]:
current_color_bound_setting = 1.0
for subdir, dirs, files in os.walk(data_dir):
    if 'notUsable' in subdir:
        continue
    if 'mm_hidden' in subdir:
        continue
    if 'converted_zda' in subdir:
        continue
    
    zda_file = None
    # locate zda file
    for f in files:
        if f.endswith('.zda'):
            zda_file = f
        else:
            continue

        rec_id = zda_file.split('.')[0]
        print(rec_id)
        # movie dir
        movie_dir = subdir + "/movie" + rec_id + "/"
        try:
            os.makedirs(movie_dir)
        except Exception as e:
            pass

        # determine if we are even missing any jpg'd frames
        need_to_open_zda = True
        for frame in range(start_frame, start_frame + movie_length):
            filename = movie_dir + str(frame) + ".jpg"
            if not os.path.exists(filename):
                need_to_open_zda = True
                break
        if not need_to_open_zda and not replace_existing_frame_pngs:
            continue
            
        recording_max_snr = 20.0  # gets changed
        
        if not disable_photoZ_interact:
            aPhz = AutoPhotoZ(data_dir=subdir)
            # open the PhotoZ file
            aPhz.select_PhotoZ()

            print("\n\nOpening", zda_file)
            aPhz.open_zda_file(subdir + "/" + zda_file)

            # estimate the SNR max across all frames
            aPhz.set_measure_window(start_frame, movie_length)
            filename = aPhz.save_background()
            if filename is not None:  # otherwise, sticks with default rec max of 20
                
                # open Data.dat and read in the array max
                intermed_snr_df = pd.read_csv(filename, 
                                              sep='\t', 
                                              header=None, 
                                              names=['Index',  'Values'])
                recording_max_snr = intermed_snr_df['Values'].max()

                recording_max_snr *= 1.05  # 5% higher for headroom
                recording_max_snr = round(recording_max_snr, 2)
                if clean_scratch_files:
                    try:
                        os.remove(filename)
                    except Exception as e:
                        print("Could not remove", filename)
                        print(e)
            
            # set the normalization to be consistent across entire video
            aPhz.set_color_upper_bound(recording_max_snr, current_color_bound_setting)
            print("Moved color upper bound from", current_color_bound_setting, "to", recording_max_snr)
            current_color_bound_setting = recording_max_snr
            
            # set measure window width to 1
            aPhz.set_measure_window(None, 1)
        
        ##### Get frame annotation: layer and stim markings, and store #####
            
        # file conventions across
        electrode_file = subdir + "/electrode.dat"
        corners_layer_file = subdir + "/corners_layer.dat"
        corners_barrel_file = subdir + "/corners_barrel.dat"
        image_border_width = 8
        image_scaling = 7  # pixels of image per array diode pixel
        electrode_marker_radius = 4
        annotations_enabled = all([os.path.exists(f) for f in [electrode_file, 
                                                               corners_layer_file, 
                                                               corners_barrel_file]])

        def array_point_to_image_pixel(arr_pt):
            x_arr, y_arr = arr_pt
            return [image_border_width + image_scaling * x_arr, 
                    image_border_width + image_scaling * y_arr]

        if annotations_enabled:
            # read in electrode location
            with open(electrode_file, 'r') as f:
                lines = f.readlines()
            stim_pt = int(lines[-1]) # last line is always electrode location
            aux_obj = LaminarROI([stim_pt]).get_points()
            stim_pt = aux_obj[0]  # should be a list of len 2, representing px location [x, y]
            print("Stim point:", stim_pt)
            stim_img_px = array_point_to_image_pixel(stim_pt)
            stim_xy = (stim_img_px[0] - electrode_marker_radius, 
                       stim_img_px[1] - electrode_marker_radius, 
                       stim_img_px[0] + electrode_marker_radius, 
                       stim_img_px[1] + electrode_marker_radius)
            # read in corners files
            la_corners = {corners_layer_file: {},
                          corners_barrel_file: {}}
            for la_file in [corners_layer_file, corners_barrel_file]:
                with open(la_file, 'r') as f:
                    lines = f.readlines() 
                corners = [int(x) for x in lines[4:]] # the last 4 lines are diode numbers of corners
                layer_axes = LayerAxes(corners)
                laminar_axis, laminar_axis_2 = layer_axes.get_layer_axes()
                print("laminar axis unit vectors:", laminar_axis.get_unit_vector(), 
                      laminar_axis_2.get_unit_vector())

                end = array_point_to_image_pixel(laminar_axis.get_end_point())
                end2 = array_point_to_image_pixel(laminar_axis_2.get_end_point())
                start = array_point_to_image_pixel(laminar_axis.get_start_point())
                start2 = array_point_to_image_pixel(laminar_axis_2.get_start_point())
                la_corners[la_file]["line_loc"] = [start + end, start2 + end2]
                la_corners[la_file]["text_loc"] = (int((end[0] + end2[0]) / 2), 
                                                   int((end[1] + end2[1]) / 2))

                
        images = []
        img_filenames = []
        for frame in range(start_frame, start_frame + movie_length):
            filename = movie_dir + str(frame) + ".jpg"
            if replace_existing_frame_pngs or not os.path.exists(filename):
                # change frame
                if not disable_photoZ_interact:
                    aPhz.set_measure_window(frame, None)

                    # export this frame
                    aPhz.save_map_jpeg(filename)
                    print("File created:", filename)
                    
            if annotations_enabled:
                # draw electrode location on image <filename>
                image = Image.open(filename)
                draw = ImageDraw.Draw(image)
                draw.ellipse(stim_xy, outline='red', width=2)
                # read in corners files
                for la_file in [corners_layer_file, corners_barrel_file]:
                    draw.line(la_corners[la_file]["line_loc"][0], fill='white', width=2)
                    draw.line(la_corners[la_file]["line_loc"][1], fill='white', width=2)
                    if la_file == corners_layer_file:
                        draw.text(la_corners[la_file]["text_loc"], "L4", anchor="ms")
                # PIL image write back
                image.save(filename)

            try:
                images.append(imageio.imread(filename))
                img_filenames.append(filename)
            except Exception as e:
                pass
        
        # create gif
        created_movie = False
        try:
            imageio.mimsave(movie_dir + rec_id + 'movie.gif', images)
            print("CREATED MOVIE:", rec_id + 'movie.gif')
            created_movie = True
            if clean_scratch_files:
                for img_file in img_filenames:
                    os.remove(img_file)
        except Exception as e:
            if not created_movie:
                print("Not creating movie for " + rec_id)
            print(e)

02_01_05


Opening 02_01_05.zda
Moved color upper bound from 1.0 to 18.52
Stim point: [8, 61]
edge_pts [[24, 0], [51, 0]] axis_pts [[1, 55], [20, 71]]
laminar axis unit vectors: [0.3858061304424809, -0.922579877145063] [0.40014159439896263, -0.9164533291073015]
edge_pts [[19, 79], [63, 79]] axis_pts [[5, 68], [17, 41]]
laminar axis unit vectors: [0.7863183388224225, 0.6178215519319035] [0.7709617516270895, 0.6368814469962913]
CREATED MOVIE: 02_01_05movie.gif
03_01_06


Opening 03_01_06.zda
Moved color upper bound from 18.52 to 9.82
Stim point: [34, 55]
edge_pts [[31, 79], [64, 79]] axis_pts [[23, 58], [51, 50]]
laminar axis unit vectors: [0.3559953275919878, 0.9344877349289681] [0.40905590409290915, 0.9125093245149513]
edge_pts [[31, 79], [64, 79]] axis_pts [[23, 58], [51, 50]]
laminar axis unit vectors: [0.3559953275919878, 0.9344877349289681] [0.40905590409290915, 0.9125093245149513]
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-10-27\03-01-06_L4_in

CREATED MOVIE: 02_01_02movie.gif
02_01_08


Opening 02_01_08.zda
Moved color upper bound from 12.0 to 7.88
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\03-01-08_aka_02-01-08_L23_intralaminar/movie02_01_08/94.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\03-01-08_aka_02-01-08_L23_intralaminar/movie02_01_08/95.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\03-01-08_aka_02-01-08_L23_intralaminar/movie02_01_08/96.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\03-01-08_aka_02-01-08_L23_intralaminar/movie02_01_08/97.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\03-01-08_aka_02-01-08_L23_intralaminar/movie02_01_08/98.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\03-01-08_aka_02-01-08_L23_intralaminar/movie02_01_08/99.jpg
File create



Opening 03_01_01.zda
Moved color upper bound from 8.73 to 8.65
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\06-01-01_aka_part2_03-01-01_L4_interlaminar/movie03_01_01/94.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\06-01-01_aka_part2_03-01-01_L4_interlaminar/movie03_01_01/95.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\06-01-01_aka_part2_03-01-01_L4_interlaminar/movie03_01_01/96.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\06-01-01_aka_part2_03-01-01_L4_interlaminar/movie03_01_01/97.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\06-01-01_aka_part2_03-01-01_L4_interlaminar/movie03_01_01/98.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\06-01-01_aka_part2_03-01-01_L4_interlaminar/movie03_01_01/99.jpg
File created: C:/Users/

File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\08-01-04_aka_part2_05-01-04_L4_interlaminar/movie05_01_04/120.jpg
CREATED MOVIE: 05_01_04movie.gif
07_01_01


Opening 07_01_01.zda
Moved color upper bound from 10.24 to 9.15
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\10-01-01_aka_part2_07-01-01_L4_interlaminar/movie07_01_01/94.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\10-01-01_aka_part2_07-01-01_L4_interlaminar/movie07_01_01/95.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\10-01-01_aka_part2_07-01-01_L4_interlaminar/movie07_01_01/96.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\10-01-01_aka_part2_07-01-01_L4_interlaminar/movie07_01_01/97.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\10-01-01_aka_part2_07-01-01_L4_interlaminar/

File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\11-01-06_aka_part2_08-01-06_L4_interlaminar/movie08_01_06/119.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-28\11-01-06_aka_part2_08-01-06_L4_interlaminar/movie08_01_06/120.jpg
CREATED MOVIE: 08_01_06movie.gif
04_01_01


Opening 04_01_01.zda
Moved color upper bound from 10.19 to 7.95
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-29\04-01-01_L4_interlaminar/movie04_01_01/94.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-29\04-01-01_L4_interlaminar/movie04_01_01/95.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-29\04-01-01_L4_interlaminar/movie04_01_01/96.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2020-12-29\04-01-01_L4_interlaminar/movie04_01_01/97.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pi

...

File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-26\07-01-03_L4_interlaminar_intralaminar/movie07_01_03/107.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-26\07-01-03_L4_interlaminar_intralaminar/movie07_01_03/108.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-26\07-01-03_L4_interlaminar_intralaminar/movie07_01_03/109.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-26\07-01-03_L4_interlaminar_intralaminar/movie07_01_03/110.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-26\07-01-03_L4_interlaminar_intralaminar/movie07_01_03/111.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-26\07-01-03_L4_interlaminar_intralaminar/movie07_01_03/112.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-26\07-01-03_L4_interlaminar_intral

File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-31\06-01-05_L23_interlaminar_intralaminar/movie06_01_05/108.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-31\06-01-05_L23_interlaminar_intralaminar/movie06_01_05/109.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-31\06-01-05_L23_interlaminar_intralaminar/movie06_01_05/110.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-31\06-01-05_L23_interlaminar_intralaminar/movie06_01_05/111.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-31\06-01-05_L23_interlaminar_intralaminar/movie06_01_05/112.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-31\06-01-05_L23_interlaminar_intralaminar/movie06_01_05/113.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-31\06-01-05_L23_interlaminar

File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-31\09-01-05_L23_interlaminar/movie09_01_05/107.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-31\09-01-05_L23_interlaminar/movie09_01_05/108.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-31\09-01-05_L23_interlaminar/movie09_01_05/109.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-31\09-01-05_L23_interlaminar/movie09_01_05/110.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-31\09-01-05_L23_interlaminar/movie09_01_05/111.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-31\09-01-05_L23_interlaminar/movie09_01_05/112.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-05-31\09-01-05_L23_interlaminar/movie09_01_05/113.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pip

File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-06-28\03-01-02_L5_interlaminar/movie03_01_02/113.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-06-28\03-01-02_L5_interlaminar/movie03_01_02/114.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-06-28\03-01-02_L5_interlaminar/movie03_01_02/115.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-06-28\03-01-02_L5_interlaminar/movie03_01_02/116.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-06-28\03-01-02_L5_interlaminar/movie03_01_02/117.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-06-28\03-01-02_L5_interlaminar/movie03_01_02/118.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-06-28\03-01-02_L5_interlaminar/movie03_01_02/119.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_t

File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-06-28\05-01-02_L5_interlaminar/movie05_01_02/119.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-06-28\05-01-02_L5_interlaminar/movie05_01_02/120.jpg
CREATED MOVIE: 05_01_02movie.gif
06_01_01


Opening 06_01_01.zda
Moved color upper bound from 14.27 to 15.7
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-06-28\06-01-01_L5_interlaminar/movie06_01_01/94.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-06-28\06-01-01_L5_interlaminar/movie06_01_01/95.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-06-28\06-01-01_L5_interlaminar/movie06_01_01/96.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-06-28\06-01-01_L5_interlaminar/movie06_01_01/97.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Kate's\2021-06-28\06-01

In [5]:
# copying movies into Google Drive
if not disable_google_drive_interact:
    for subdir, dirs, files in os.walk(data_dir):
        if 'notUsable' in subdir:
            continue
        if 'mm_hidden' in subdir:
            continue
            
        extend = subdir.split("movie_targets/")[-1]
        if "movie" in extend:
            
            dst_dir = google_drive_dir + extend
            if os.path.exists(dst_dir):
                try:
                    shutil.rmtree(dst_dir, ignore_errors=False, onerror=None)
                except Exception as e:
                    print("Dest dir", dst_dir, "not deleted. Skipping.")
                    print(e)
                    continue

            print(subdir, "\n", dst_dir, "\n\n")
            shutil.copytree(subdir, dst_dir)


In [6]:
# clean up local drive
if not disable_google_drive_interact:
    for subdir, dirs, files in os.walk(data_dir):
        if 'notUsable' in subdir:
            continue

        if "movie" in subdir:
            try:
                shutil.rmtree(subdir, ignore_errors=False, onerror=None)
            except Exception as e:
                print("Dir", subdir, "not deleted. Skipping.")
                print(e)
                continue

            print("Deleted:", subdir)